# MO412A - Project

In [8]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
from tqdm import tqdm

In [9]:
# Get all the links

link_parts = [
    'https://www25.senado.leg.br/web/atividade/materias?p_p_id=materia_WAR_atividadeportlet',
    'p_p_lifecycle=0',
    '_materia_WAR_atividadeportlet_pesquisaAvancada=true',
    '_materia_WAR_atividadeportlet_ano=',
    '_materia_WAR_atividadeportlet_tipoAutor=SENADOR',
    '_materia_WAR_atividadeportlet_dataApresentacaoInicial={}',
    '_materia_WAR_atividadeportlet_numero=',
    '_materia_WAR_atividadeportlet_situacaoTramitacao=TRAMITACAO_ENCERRADA',
    '_materia_WAR_atividadeportlet_dataApresentacaoFinal={}',
    '_materia_WAR_atividadeportlet_anoNorma=',
    '_materia_WAR_atividadeportlet_codigoClasseTematica=',
    '_materia_WAR_atividadeportlet_palavraChave=',
    '_materia_WAR_atividadeportlet_tipoNorma=',
    '_materia_WAR_atividadeportlet_codigoParlamentar=',
    '_materia_WAR_atividadeportlet_numeroNorma=',
    '_materia_WAR_atividadeportlet_nomeRelator=',
    '_materia_WAR_atividadeportlet_btnSubmit=',
    '_materia_WAR_atividadeportlet_nomeAutor=',
    '_materia_WAR_atividadeportlet_codigoTipoConteudo=',
    '_materia_WAR_atividadeportlet_siglasTipos=DESTPEC%2CDESTPLORD',
    '_materia_WAR_atividadeportlet_p={}'
]

link = '&'.join(link_parts)

In [10]:
def get_data(link: str, start_date: datetime.date, end_date: datetime.date, page: int):
    formatted_link = link.format(start_date.strftime('%d%%2F%m%%2F%Y'), end_date.strftime('%d%%2F%m%%2F%Y'), page)
    response = requests.get(formatted_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [11]:
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2024, 5, 1)
page=1 # Start at page 1

document_links = {}
while True:
    soup = get_data(link, start_date, end_date, page)
    result = soup.find('div', class_='sf-proposicoes-resultado div-zebra')
    if result is None:
        break
    for div in result.children:
        pl_link = div.find('a')
        if pl_link != -1:
            document_links[pl_link.text] = pl_link['href']
    page += 1

In [12]:
df = pd.DataFrame(document_links, index=[0]).T.reset_index()
df.columns = ['document', 'link']
df.to_csv(f'document_links_from_{start_date}_to_{end_date}.csv', index=False)